In [2]:
search_for = 132
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004065752029418945
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 1268606955
type: [1, 1, 1, 1, 132] 132
cases of this type: 303595776
100000 46.09869733504245
200000 92.24069225841752
300000 99.47873933780214
400000 99.78246195774562
500000 98.40369420876296
600000 101.98386334019517
700000 99.16025737622552
800000 96.51060534931203
900000 96.7695057721601
1000000 98.41674749992649
1100000 96.8667418985055
1200000 25.712204453416913
1300000 96.13675280649423
1400000 96.82098771108318
1500000 95.56638913650723
1600000 96.67427508514088
1700000 96.452498627175
1800000 101.55803209726004
1900000 96.86615072624093
2000000 96.28685891066607
2100000 96.5209682582852
2200000 97.67774760124607
2300000 75.65458124291497
2400000 46.0173664945794
2500000 93.98567678132463
2600000 92.31769403332926
2700000 90.98145280979301
2800000 91.60254366603613
2900000 94.65461557706135
3000000 93.38515738970574
31

29700000 68.63940750021057
29800000 74.3622695308763
29900000 105.80484578491924
30000000 98.93897836410375
30100000 97.48330614088
30200000 41.23223908638494
30300000 100.2425659234013
30400000 99.86649534481425
30500000 98.55806497235453
30600000 95.83324752985267
30700000 97.39986379599121
30800000 93.61400448528653
30900000 93.05018844592446
31000000 89.99133853619556
31100000 26.002867703529866
31200000 95.61145305668055
31300000 93.46345365640911
31400000 92.80347831948336
31500000 97.68287886855235
31600000 98.09639438356008
31700000 97.20194580175962
31800000 98.40114731483648
31900000 98.22579449336928
32000000 55.39710232135302
32100000 85.6730811914349
32200000 102.44290861719364
32300000 102.00741068925157
32400000 106.7778629673418
32500000 33.37533322409785
32600000 106.06681769274044
32700000 103.59907887715971
32800000 101.20645036520222
32900000 103.59277613374648
33000000 101.57758131269495
33100000 99.70012556985306
33200000 97.4423962390921
33300000 93.3397577794551

60000000 99.94311958092511
60100000 104.045658722668
60200000 101.81231759543121
60300000 36.61066281248417
60400000 97.81293062927874
60500000 95.86243872456534
60600000 95.88371774114536
60700000 92.5658445205639
60800000 90.87119833197347
60900000 90.36231234150583
61000000 23.2329412473291
61100000 94.34743915199462
61200000 91.94992184631808
61300000 91.85226463024372
61400000 89.04577108243292
61500000 95.50181849232344
61600000 97.60658589003114
61700000 36.55246817388036
61800000 103.5286386837722
61900000 102.24991600615975
62000000 99.87714170195389
62100000 103.70964799641354
62200000 95.78876612883053
62300000 95.13453232354883
62400000 99.28479458009848
62500000 96.1369347357499
62600000 49.343163283444305
62700000 80.38370298121998
62800000 90.46203589194123
62900000 88.10175094768526
63000000 81.71027462462526
63100000 84.83977236871237
63200000 86.75636196793023
63300000 22.381152349948586
63400000 89.47839684557971
63500000 87.06057814876998
63600000 82.33209262852809


90400000 47.021303101933206
90500000 57.23682794501862
90600000 72.21375828142433
90700000 80.15651337516539
90800000 76.03247454903382
90900000 21.214195848189046
91000000 83.10014933545406
91100000 78.6069528513198
91200000 69.27714363847599
91300000 68.03451587315034
91400000 43.03626276391881
91500000 77.21434783008334
91600000 81.85097338301803
91700000 82.20965202803974
91800000 87.08578579489442
91900000 82.11357896261507
92000000 89.68181097033722
92100000 85.94830269844105
92200000 86.66991334646366
92300000 93.86959080011941
92400000 84.37018694514691
92500000 85.41120872826384
92600000 79.1280336872716
92700000 52.84774297125583
92800000 52.80684969717538
92900000 80.66703798562786
93000000 82.00662587807474
93100000 79.28217325044906
93200000 18.5858503951459
93300000 78.70537732971566
93400000 83.03538476126774
93500000 80.77895139171416
93600000 64.32632219164839
93700000 50.66339717158318
93800000 77.99702459201185
93900000 83.09208609050417
94000000 71.63830941685129
94

120000000 73.34304311691602
120100000 79.48146315366066
120200000 77.31023347719054
120300000 78.42340294491729
120400000 75.80111225380763
120500000 32.463485718228426
120600000 23.665666368249436
120700000 41.017401016851814
120800000 15.702952130339353
120900000 63.52888523489088
121000000 48.31473738242195
121100000 56.9097445119306
121200000 57.490339380467624
121300000 48.4656601987891
121400000 53.93200858262199
121500000 41.14346705389035
121600000 40.71198456113694
121700000 53.04619473500727
121800000 51.18413200959391
121900000 70.04592942950106
122000000 71.90685349039475
122100000 49.488692851030855
122200000 40.655802753772356
122300000 50.544005783668695
122400000 50.70665985283832
122500000 64.12198866561029
122600000 50.61253799970592
122700000 38.56577754127634
122800000 43.47571480251288
122900000 21.44375572313375
123000000 48.292986346885954
123100000 10.150041268161111
123200000 43.841268325379396
123300000 23.26229764819321
123400000 37.49547528322435
123500000 3

...

1189800000 5.328144613820195
1189900000 2.8682056244596508
1190000000 2.554618337159713
1190100000 3.551741334087253
1190200000 2.955137290834927
1190300000 3.201395549986227
1190400000 4.126362489041984
1190500000 3.705454393027458
1190600000 2.9117647957702375
1190700000 3.5604806298773584
1190800000 3.7393335640108156
1190900000 2.751742707545324
1191000000 2.950749511043221
1191100000 3.280949389092584
1191200000 3.268502210203803
1191300000 3.149603144654403
1191400000 3.391126250531104
1191500000 3.166565994650029
1191600000 4.184221001691242
best so far: 0
type: [1, 4, 1, 3, 11] 132
cases of this type: 1581228
1191700000 2.2833746820063725
1191800000 1.4962669795835084
1191900000 1.0793858520766795
1192000000 1.6410069205764304
1192100000 1.7021841858300863
1192200000 1.257678832446436
1192300000 1.3923685388928289
1192400000 1.678905578295029
1192500000 1.4148924527734617
1192600000 1.337913531924741
1192700000 1.7381079906751515
1192800000 1.5212373737770915
1192900000 1.12487

1203600000 3.7137215181860626
1203700000 3.931088146239324
1203800000 3.8246749760852983
1203900000 3.896745168011089
1204000000 3.3582165118482483
1204100000 2.670779293420997
1204200000 4.180742757503241
1204300000 3.6635314263297087
1204400000 2.81132647563581
1204500000 3.9798740064754488
1204600000 3.3736880335988535
1204700000 2.5979229914725224
1204800000 4.103776543560793
1204900000 3.0906440216253968
1205000000 3.6942962652799354
1205100000 4.110347248254051
1205200000 2.6992846478854022
1205300000 3.4298137374446593
1205400000 4.082387224154145
1205500000 2.630490360404114
1205600000 3.592275846257683
1205700000 3.783073486862381
1205800000 2.3159635142121813
1205900000 3.59118684562554
1206000000 3.8182371290306647
1206100000 2.2920777986458405
1206200000 3.739178312815984
1206300000 3.5875573973530233
1206400000 2.0598853168203664
1206500000 3.8939135862312377
1206600000 3.676818569421341
1206700000 2.687455076579167
1206800000 4.338966468647285
1206900000 3.402933636431982

1230800000 0.7752200878485143
1230900000 1.2146598523190617
1231000000 1.1988030987105005
1231100000 0.7032914489082925
1231200000 1.19070057094121
1231300000 1.1712838318095837
1231400000 0.7706678621870975
1231500000 1.1797453388934733
1231600000 1.1158780798651973
1231700000 0.7985302176555992
1231800000 1.284086503317982
1231900000 1.145512165819039
1232000000 0.8114294908889399
1232100000 1.2095753464512626
1232200000 0.8997079234142402
1232300000 0.7401935408379667
1232400000 1.2748354528235495
1232500000 1.0714579964850843
1232600000 1.0288449594347013
1232700000 1.1386371342502932
1232800000 0.8767716141033437
1232900000 0.9502668653684391
best so far: 0
type: [2, 1, 1, 6, 11] 132
cases of this type: 3162456
1233000000 1.2263755550211768
1233100000 0.6286638107335071
1233200000 0.9981352231107685
1233300000 1.0499905792397262
1233400000 0.6085863744183746
1233500000 0.8992712808888489
1233600000 1.0132498235917886
1233700000 0.6044413078128266
1233800000 1.0126993434922829
1233

1252400000 0.47449426566551794
1252500000 0.4960777998019059
1252600000 0.5314821574489142
1252700000 0.4523938942374819
1252800000 0.47556558957584705
1252900000 0.5365152565407852
1253000000 0.44685354617211226
1253100000 0.47696431986805793
1253200000 0.45146615110622845
best so far: 0
type: [2, 2, 1, 3, 11] 132
cases of this type: 790614
1253300000 0.3255774164091381
1253400000 0.26551644333950675
1253500000 0.20653401723829243
1253600000 0.24624822652330003
1253700000 0.25189861769628524
1253800000 0.24630445957249075
1253900000 0.23990122072059578
1254000000 0.23186266402912137
best so far: 0
type: [2, 2, 1, 11, 3] 132
cases of this type: 215622
1254100000 0.26853460192794604
1254200000 0.23009240484366816
best so far: 0
type: [2, 2, 1, 33, 1] 132
cases of this type: 71874
1254300000 0.1412255643125077
best so far: 0
type: [2, 2, 3, 1, 11] 132
cases of this type: 263538
1254400000 0.23979025877984694
1254500000 0.15374494829593763
best so far: 0
type: [2, 2, 3, 11, 1] 132
cases o

best so far: 0
type: [3, 2, 11, 1, 2] 132
cases of this type: 3872
best so far: 0
type: [3, 2, 11, 2, 1] 132
cases of this type: 1936
best so far: 0
type: [3, 2, 22, 1, 1] 132
cases of this type: 968
best so far: 0
type: [3, 4, 1, 1, 11] 132
cases of this type: 58564
best so far: 0
type: [3, 4, 1, 11, 1] 132
cases of this type: 5324
best so far: 0
type: [3, 4, 11, 1, 1] 132
cases of this type: 484
best so far: 0
type: [3, 11, 1, 1, 4] 132
cases of this type: 2816
best so far: 0
type: [3, 11, 1, 2, 2] 132
cases of this type: 1408
best so far: 0
type: [3, 11, 1, 4, 1] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 1, 2] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 2, 1] 132
cases of this type: 352
best so far: 0
type: [3, 11, 4, 1, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 1, 1, 2] 132
cases of this type: 352
best so far: 0
type: [3, 22, 1, 2, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 2, 1, 1] 132
cases of this type: 88
best